In [18]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer, util
from torch import embedding
from nltk.corpus import stopwords
from nltk.corpus import wordnet 
import nltk
import numpy as np
from sklearn import preprocessing
import math

In [19]:
#subject6 的最明顯波長是68.4813s
#space = 17.12 #sample step size
#timeframe = 17.12
global q
q=0

In [20]:
import nltk
nltk.download('omw-1.4')

[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\chang\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [21]:
class Subject():
    def __init__(self,subject):
        self.subject = subject

# 算所有的值

In [22]:
from tracemalloc import start


def calculate(gcs,destination,timeframe,space,start_time=0,end=480): #所以gcs要是string

    
    df = pd.read_csv(gcs) 

    df['Start_time'] = pd.to_numeric(df['Start_time'])
    #df['end_time'] = pd.to_numeric(df['end_time'])
    global all_sentences001
    all_sentences001 = []
    t=start_time
    
    while t<end:
        sentences = []
        for index, rows in df.loc[(df["Start_time"]>=t) & (df["Start_time"]<=(t+timeframe))].iterrows():
            sentences.append(rows["Words"])
        all_sentences001.append(" ".join(sentences))
        t+=space

    
    model = SentenceTransformer('all-MiniLM-L6-v2')
    global SBERT 
    SBERT= []

    def get_cosine_similarity(feature_vec_1, feature_vec_2):    
        return cosine_similarity(feature_vec_1.reshape(1, -1), feature_vec_2.reshape(1, -1))[0][0]
    # Two lists of sentences
    for i in range(0,len(all_sentences001)-1):
        #Compute embedding for both lists
        embeddings1 = model.encode(all_sentences001[i], convert_to_tensor=True)
        #print(all_sentences001[i])
        embeddings2 = model.encode(all_sentences001[i+1], convert_to_tensor=True)
        #print(all_sentences001[i+1])
        #Compute cosine-similarities
        cosine_scores = util.cos_sim(embeddings1, embeddings2)
        #print(cosine_scores)
        # convert from PyTorch tensor to numpy array
        SBERT.append(get_cosine_similarity(embeddings1, embeddings2))
    ######
        '''print("timeframe: ",i,":",i+10,"\n")
        print(get_cosine_similarity(embeddings1, embeddings2),"\n")
        print(all_sentences001[i],"\n")
        print(all_sentences001[i+1],"\n")
        print()'''

    
    all_sentences001 = []
    stops = set(stopwords.words('english'))
    t=start_time
    while t <end:
        sentences = []
        for index, rows in df.loc[(df["Start_time"]>=t) & (df["Start_time"]<=(t+timeframe))].iterrows():
            if rows["Words"].lower() not in stops:
                sentences.append(rows["Words"].lower())
                #print(rows["Words"])
        all_sentences001.append(" ".join(sentences))
        #print(t)
        t += space 
    ''' 舊的sample step
    for t in range(0,480,0.5):
        sentences = []
        for index, rows in df.loc[(df["Start_time"]>=t) & (df["Start_time"]<=(t+10))].iterrows():
            if rows["Words"].lower() not in stops:
                sentences.append(rows["Words"].lower())
        all_sentences001.append(" ".join(sentences))
    '''    
    #nltk.download('omw-1.4')
    #nltk.download('wordnet')
    #nltk.download('stopwords')
    global ancestor_depth_list
    ancestor_depth_list = []
    global ancestors_depth
    ancestors_depth = []
    global pair_counts
    pair_counts = []
    stops = set(stopwords.words('english'))
    global all_sentences_wu_sum
    all_sentences_wu_sum = []
    global all_sentences_wu_sum_divided
    all_sentences_wu_sum_divided = []
    Timeframe_percentage = []  #每個timeframe的valid pair percentage的list
    for sentence_num in range(0,len(all_sentences001)): #現在第幾句
        wu_pulmer_sum = 0
        validPairs = 0
        the_sentence = str(all_sentences001[sentence_num]).split()
        #print(the_sentence)
        timeframe_pairs_count = 0
        localMin_depth = 0
        for words_num in range(0, (len(the_sentence)-1)): #標定第一個字
            if len(wordnet.synsets(the_sentence[words_num])) > 0 :
                syns1 = wordnet.synsets(the_sentence[words_num])
                syns1_num = len(syns1)
            else:
                continue
            for words_num2 in range((words_num+1), len(the_sentence)): #標定第二個字
                
                hasAncestors = True
                if len(wordnet.synsets(the_sentence[words_num2])) > 0 :
                    syns2 = wordnet.synsets(the_sentence[words_num2])
                    syns2_num = len(syns2)
                    for i in range(0, syns1_num):  # 標定第一個synset
                        for u in range(0, syns2_num): #標定第二個synset
                            #print(the_sentence[words_num],"的",syns1[i]," ", the_sentence[words_num2],"的",syns2[u]," " ,syns1[i].wup_similarity(syns2[u]))
                            wu_pulmer_sum += syns1[i].wup_similarity(syns2[u])
                            timeframe_pairs_count += 1
                            if syns1[i].lowest_common_hypernyms(syns2[u]) != []:
                                #localAncestors[sentence_num].append(syns1[i].lowest_common_hypernyms(syns2[u])[0])
                                localMin_depth+=(syns1[i].lowest_common_hypernyms(syns2[u])[0].min_depth())
                                hasAncestors = True
                else:
                    continue
            if hasAncestors:
                validPairs += 1
        #這邊是跑完一句話的處理
        if timeframe_pairs_count != 0:
            all_sentences_wu_sum_divided.append(wu_pulmer_sum/timeframe_pairs_count) #算出除以pairs數量的平均值
        else: 
            all_sentences_wu_sum_divided.append(wu_pulmer_sum) #此時wu-palmer sum就會是0
        all_sentences_wu_sum.append(wu_pulmer_sum)
        pair_counts.append(timeframe_pairs_count)
        #print(all_sentences_wu_sum)
        global wu_difference
        wu_difference = []
        for x in range(len(all_sentences_wu_sum)-1):
            wu_difference.append(all_sentences_wu_sum[x+1]-all_sentences_wu_sum[x])
    # 計算ancestors' depth
        if timeframe_pairs_count != 0:
            ancestor_depth_list.append(localMin_depth/timeframe_pairs_count)
        else:
            ancestor_depth_list.append(localMin_depth) #此時會是0
    '''for i in range(len(localAncestors)):
        localMin_depth = 0
        for u in range(len(localAncestors[i])):
            localMin_depth += localAncestors[i][u].min_depth() #一句話所有min depth加起來
        ancestors_depth.append((localMin_depth/len(localAncestors[i])))'''
        #Timeframe_percentage += validPairs/(len(the_sentence)*(len(the_sentence)-1))
        #print(Timeframe_percentage)
    results_csv = pd.DataFrame(list(zip(SBERT,all_sentences_wu_sum,all_sentences_wu_sum_divided,pair_counts,ancestor_depth_list)), columns = ['SBERT',"Wu-Pulmer","Wu-Pulmer_divided","pair_counts","ancestor_depth"])
    results_csv.to_csv(f"data/Text/scan_results/shuffled_scan_data/{q:0>3d}({destination}).csv")
    
    #print(timeframe_pairs_count)
    
    """ # for normalization
    wu_difference = np.array(wu_difference)
    SBERT = np.array(SBERT)
    #normalized_SBERT = preprocessing.normalize([SBERT])
    #normalized_wu_difference = preprocessing.normalize([wu_difference])


    min_max_scaler = preprocessing.MinMaxScaler()
    SBERT = np.reshape(SBERT,(479,1))
    wu_difference = np.reshape(wu_difference,(479,1))
    global normalized_SBERT
    global normalized_wu_difference
    normalized_SBERT  = min_max_scaler.fit_transform(SBERT)
    normalized_wu_difference  = min_max_scaler.fit_transform(wu_difference)
    """


# 算common ancestor depth

In [23]:
def common_depth_calculate(gcs,timeframe,space,start_time=0,end=480): #所以gcs要是string


    df = pd.read_csv(gcs) 

    df['Start_time'] = pd.to_numeric(df['Start_time'])
    #df['end_time'] = pd.to_numeric(df['end_time'])
    global all_sentences001
    all_sentences001 = []
    t=start_time

    while t<end:
        sentences = []
        for index, rows in df.loc[(df["Start_time"]>=t) & (df["Start_time"]<=(t+timeframe))].iterrows():
            sentences.append(rows["Words"])
        all_sentences001.append(" ".join(sentences))
        t+=space


    all_sentences001 = []
    stops = set(stopwords.words('english'))
    t=start_time
    while t <end:
        sentences = []
        for index, rows in df.loc[(df["Start_time"]>=t) & (df["Start_time"]<=(t+timeframe))].iterrows():
            if rows["Words"].lower() not in stops:
                sentences.append(rows["Words"].lower())
                #print(rows["Words"])
        all_sentences001.append(" ".join(sentences))
        #print(t)
        t += space 
    ''' 舊的sample step
    for t in range(0,480,0.5):
        sentences = []
        for index, rows in df.loc[(df["Start_time"]>=t) & (df["Start_time"]<=(t+10))].iterrows():
            if rows["Words"].lower() not in stops:
                sentences.append(rows["Words"].lower())
        all_sentences001.append(" ".join(sentences))
    '''   
    #nltk.download('omw-1.4')
    #nltk.download('wordnet')
    #nltk.download('stopwords')
    global actual_pair_counts
    actual_pair_counts = 0
    stops = set(stopwords.words('english'))
    global all_sentences_wu_sum
    all_sentences_wu_sum = []
    global all_sentences_wu_sum_divided
    all_sentences_wu_sum_divided = []
    global common_ancestors 
    common_ancestors = []
    global word_pair_counts
    word_pair_counts = 0
    global common_ancestor_synset_average_depth 
    common_ancestor_synset_average_depth  = 0
    global ancestor_depth_list
    ancestor_depth_list = []
    valid_word_pair_counts = 0 #計算有common ancestors的word pair
    for sentence_num in range(0,len(all_sentences001)): #現在第幾句
        the_sentence = str(all_sentences001[sentence_num]).split()
        #print(the_sentence)
        timeframe_pairs_count = 0
        for words_num in range(0, (len(the_sentence)-1)): #標定第一個字的
            if len(wordnet.synsets(the_sentence[words_num])) > 0 :
                syns1 = wordnet.synsets(the_sentence[words_num])
                syns1_num = len(syns1)
            
            else:
                continue
            for words_num2 in range((words_num+1), len(the_sentence)): #標定第二個字
                valid_pair_indicator = False
                word_pair_counts += 1
                if len(wordnet.synsets(the_sentence[words_num2])) > 0 :
                    syns2 = wordnet.synsets(the_sentence[words_num2])
                    syns2_num = len(syns2)
                    for i in range(0, syns1_num): #標定第一個字的synset
                        for u in range(0, syns2_num): #標定第二個字的synset
                            #print(the_sentence[words_num],"的",syns1[i]," ", the_sentence[words_num2],"的",syns2[u]," " ,syns1[i].wup_similarity(syns2[u]))
                            #print(syns1[i],"+",syns2[u])
                            if syns1[i].lowest_common_hypernyms(syns2[u]) != []:
                                common_ancestors.append(syns1[i].lowest_common_hypernyms(syns2[u])) 
                                ancestor_depth_list += [syns1[i].lowest_common_hypernyms(syns2[u])[0].min_depth()]
                                actual_pair_counts += 1
                                valid_pair_indicator = True
                            else:
                                ancestor_depth_list += [1] #把毫不相干的兩個字當作建立在depth = 1的ancestor上面
                                actual_pair_counts += 1
                                #print(syns1[i].lowest_common_hypernyms(syns2[u]))
                            common_ancestor_synset_average_depth += ancestor_depth_list[i*syns2_num+u]/(syns1_num*syns2_num)#這邊還在修改中，先算出任一synset ancestor depth/兩個字總synset對數
                    if valid_pair_indicator == True:
                        valid_word_pair_counts += 1
                else:
                    continue
        
    
        #print(all_sentences_wu_sum)
    
    common_ancestors_depth = 0
    for i in range(len(common_ancestors)):
        common_ancestors_depth += common_ancestors[i][0].min_depth() 
    #common_ancestors_depth = sum([x.min_depth() for x in common_ancestors]) #計算所有ancestor 的深度總和
    average_ancestors_depth = common_ancestors_depth/len(common_ancestors) #計算common ancestors' depth 的平均
    print("average_ancestors_depth=",average_ancestors_depth)
    print("ancestor_depth_averaged_by_word_counts(none as 1)=",common_ancestor_synset_average_depth/word_pair_counts )
    print("synset pairs with ancestors=", len(common_ancestors),", actual synset pairs=", actual_pair_counts, ", percentage_of_ancestors", len(common_ancestors)/actual_pair_counts
            ,", word pairs=",word_pair_counts," ,valid_wp_percentage=",valid_word_pair_counts/word_pair_counts,"\n")

# 算相對應其他區域的common ancestor 

In [24]:
def relative_depth_calculate(subject_num,destination = "ancestor_comparison", timeframe=10, space=1,start_time=0,end=480): #所以gcs要是string

    df = pd.read_csv("data/Text/{0:0>3d}(done).csv".format(subject_num)) 

    df['Start_time'] = pd.to_numeric(df['Start_time'])
    #df['end_time'] = pd.to_numeric(df['end_time'])
    #這邊是算SBERT的senteces


    global all_sentences001
    all_sentences001 = []
    stops = set(stopwords.words('english'))
    t=start_time
    while t <end:
        sentences = []
        for index, rows in df.loc[(df["Start_time"]>=t) & (df["Start_time"]<=(t+timeframe))].iterrows():
            if rows["Words"].lower() not in stops:
                sentences.append(rows["Words"].lower())
                #print(rows["Words"])
        all_sentences001.append(" ".join(sentences))
        #print(t)
        t += space 
    # 這邊把整個timezone的字加進來
    compared_words = []
    for index, rows in df.loc[(df["Start_time"]>=start_time) & (df["Start_time"]<=end)].iterrows():
            if rows["Words"].lower() not in stops:
                compared_words.append(rows["Words"].lower())
                #print(rows["Words"])

    global ancestor_depth_list
    ancestor_depth_list = []
    global ancestors_depth
    ancestors_depth = []
    global pair_counts
    pair_counts = []
    global valid_wordPairs
    valid_wordPairs = []
    stops = set(stopwords.words('english'))
    global all_sentences_wu_sum
    all_sentences_wu_sum = []
    global all_sentences_wu_sum_divided
    all_sentences_wu_sum_divided = []
    valid_word_pair_counts = 0 #計算有common ancestors的word pair
    for sentence_num in range(0,len(all_sentences001)): #現在第幾句
        the_sentence = str(all_sentences001[sentence_num]).split()
        #print(the_sentence)
        timeframe_pairs_count = 0
        wu_pulmer_sum = 0
        localMin_depth = 0
        temporary_compared_words = compared_words.copy()
        #先把compared words裡面的字去掉目前標定的句子裡的字
        for words in the_sentence:
            try: #怕沒有remove到東西會卡住
                temporary_compared_words.remove(words)
            except:
                continue
        
        for words_num in range(0, len(the_sentence)): #標定現有句子的第一個字
            validPairs = 0
            if len(wordnet.synsets(the_sentence[words_num])) > 0 :
                syns1 = wordnet.synsets(the_sentence[words_num])
                syns1_num = len(syns1)  
            else:
                continue
            for words_num2 in range(len(temporary_compared_words)): #標定temporary compared words裡面的字
                hasAncestors = False
                if len(wordnet.synsets(temporary_compared_words[words_num2])) > 0 :
                    syns2 = wordnet.synsets(temporary_compared_words[words_num2]) #裡面裝temporal words的synset
                    syns2_num = len(syns2)
                    for i in range(0, syns1_num):  # 標定第一個synset
                        for u in range(0, syns2_num): #標定第二個synset
                            #print(the_sentence[words_num],"的",syns1[i]," ", the_sentence[words_num2],"的",syns2[u]," " ,syns1[i].wup_similarity(syns2[u]))
                            wu_pulmer_sum += syns1[i].wup_similarity(syns2[u])
                            timeframe_pairs_count += 1 #算synset pairs
                            if syns1[i].lowest_common_hypernyms(syns2[u]) != []:
                                localMin_depth+=(syns1[i].lowest_common_hypernyms(syns2[u])[0].min_depth())
                                hasAncestors = True
                else:
                    continue
            if hasAncestors:
                validPairs += 1 #計算有沒有這對words有沒有任何一個synset pair有ancestor
        #這邊是跑完一句話的處理
        valid_wordPairs.append(validPairs/(len(the_sentence)*len(temporary_compared_words)))
        if timeframe_pairs_count != 0:
            all_sentences_wu_sum_divided.append(wu_pulmer_sum/timeframe_pairs_count) #算出除以pairs數量的平均值
        else: 
            all_sentences_wu_sum_divided.append(wu_pulmer_sum)
        all_sentences_wu_sum.append(wu_pulmer_sum)
        pair_counts.append(timeframe_pairs_count)
        
        #計算ancestors' depth
        ancestor_depth_list.append(localMin_depth/timeframe_pairs_count)
    results_csv = pd.DataFrame(list(zip(all_sentences_wu_sum,all_sentences_wu_sum_divided,pair_counts,ancestor_depth_list,valid_wordPairs)), columns = ["Wu-Pulmer","Wu-Pulmer_divided","pair_counts","ancestor_depth","valid word pairs"])
    
    
    results_csv.to_csv("data/Text/{0:0>3d}({1}{2}_{3}).csv".format(subject_num,destination,start_time,end))
    #print(timeframe_pairs_count)

In [25]:
'''relative_depth_calculate(8,"ancestor_comparison", 10, 1,start_time=100,end=260)
relative_depth_calculate(8,"ancestor_comparison", 10, 1,start_time=300,end=480)'''

'relative_depth_calculate(8,"ancestor_comparison", 10, 1,start_time=100,end=260)\nrelative_depth_calculate(8,"ancestor_comparison", 10, 1,start_time=300,end=480)'

In [26]:
'''x=18
startt=150
endt=270
common_depth_calculate(f"data/Text/{x:0>3d}(done).csv",endt-startt,4880,start_time=startt,end=endt)  #random_words
x=18
startt= 270
endt=480
common_depth_calculate(f"data/Text/{x:0>3d}(done).csv",endt-startt,4880,start_time=startt,end=endt)
x=18
startt= 150
endt=480
common_depth_calculate(f"data/Text/{x:0>3d}(done).csv",endt-startt,4880,start_time=startt,end=endt)'''

'x=18\nstartt=150\nendt=270\ncommon_depth_calculate(f"data/Text/{x:0>3d}(done).csv",endt-startt,4880,start_time=startt,end=endt)  #random_words\nx=18\nstartt= 270\nendt=480\ncommon_depth_calculate(f"data/Text/{x:0>3d}(done).csv",endt-startt,4880,start_time=startt,end=endt)\nx=18\nstartt= 150\nendt=480\ncommon_depth_calculate(f"data/Text/{x:0>3d}(done).csv",endt-startt,4880,start_time=startt,end=endt)'

In [27]:
'''x=6
startt=200
endt=420
common_depth_calculate(f"data/Text/{x:0>3d}(done).csv",endt-startt,4880,start_time=startt,end=endt)  #random_words
x=8
startt= 200
endt=300
common_depth_calculate(f"data/Text/{x:0>3d}(done).csv",endt-startt,4880,start_time=startt,end=endt)
x=8
startt= 300
endt=400
common_depth_calculate(f"data/Text/{x:0>3d}(done).csv",endt-startt,4880,start_time=startt,end=endt)'''

'x=6\nstartt=200\nendt=420\ncommon_depth_calculate(f"data/Text/{x:0>3d}(done).csv",endt-startt,4880,start_time=startt,end=endt)  #random_words\nx=8\nstartt= 200\nendt=300\ncommon_depth_calculate(f"data/Text/{x:0>3d}(done).csv",endt-startt,4880,start_time=startt,end=endt)\nx=8\nstartt= 300\nendt=400\ncommon_depth_calculate(f"data/Text/{x:0>3d}(done).csv",endt-startt,4880,start_time=startt,end=endt)'

In [28]:
'''x=8
startt=30
endt=130
common_depth_calculate(f"data/Text/{x:0>3d}(done_shuffled).csv",endt-startt,4880,start_time=startt,end=endt)
x=8
startt=180
endt=300
common_depth_calculate(f"data/Text/{x:0>3d}(done_shuffled).csv",endt-startt,4880,start_time=startt,end=endt)
x=8
startt=180
endt=480
common_depth_calculate(f"data/Text/{x:0>3d}(done_shuffled).csv",endt-startt,4880,start_time=startt,end=endt)
x=8
startt=200
endt=400
common_depth_calculate(f"data/Text/{x:0>3d}(done_shuffled).csv",endt-startt,4880,start_time=startt,end=endt)'''

'x=8\nstartt=30\nendt=130\ncommon_depth_calculate(f"data/Text/{x:0>3d}(done_shuffled).csv",endt-startt,4880,start_time=startt,end=endt)\nx=8\nstartt=180\nendt=300\ncommon_depth_calculate(f"data/Text/{x:0>3d}(done_shuffled).csv",endt-startt,4880,start_time=startt,end=endt)\nx=8\nstartt=180\nendt=480\ncommon_depth_calculate(f"data/Text/{x:0>3d}(done_shuffled).csv",endt-startt,4880,start_time=startt,end=endt)\nx=8\nstartt=200\nendt=400\ncommon_depth_calculate(f"data/Text/{x:0>3d}(done_shuffled).csv",endt-startt,4880,start_time=startt,end=endt)'

0.9092529740477905

參數表:<br>
all_sentences001: 所有的句子整合(沒有刪掉stop words)<br>
SBERT: 具有全部SBERT similarity的list<br>
all_sentences_wu_sum: 每句話的wu-pulmer similarity算出來的平均的list<br>
wu_difference: 兩句兩句減掉wu similarity的list<br>
normalized_SBERT: 就是表準化過後的SBERT list<br>
normalized_wu_difference: 標準化過後的wu_pulmer difference<br>
這邊注意wu-pulmer的different based應該不能這樣算

一般SD切割
halfpeakstep_phasef

timestep=1 SD切割
wholehalff


In [29]:
'''
#計算單一SUBJECT的切割
x=10
q=x
freq = float(input("enter freq: "))
period = 1/(freq/1000)
startt = input("enter start time: ")
endt= input("enter end time:")
cut_timeoffset = int(startt)
cut_timeend = int(endt)
calculate(f"data/Text/0{x}(done).csv",f"halfpeakstep_phasef{math.floor(freq)}_{startt}_{endt}",period/2,1,cut_timeoffset,cut_timeend)
'''

'\n#計算單一SUBJECT的切割\nx=10\nq=x\nfreq = float(input("enter freq: "))\nperiod = 1/(freq/1000)\nstartt = input("enter start time: ")\nendt= input("enter end time:")\ncut_timeoffset = int(startt)\ncut_timeend = int(endt)\ncalculate(f"data/Text/0{x}(done).csv",f"halfpeakstep_phasef{math.floor(freq)}_{startt}_{endt}",period/2,1,cut_timeoffset,cut_timeend)\n'

抓週期

In [30]:
def read_period(gcs): #所以gcs要是string
    df = pd.read_csv(gcs) 
    global period
    period = df["period(Wu)"][20] #現在是抓wu-pulmer divided 的週期
    #print(period)
    

def calculate(gcs,destination,timeframe,space,start_time=0,end=480): #所以gcs要是string
destination只要()裡面的名字就好

In [31]:
'''#製作一般的timeframe = 10, space = 1 的results
x=8
q=x
calculate(f"data/Text/00{x}(done_random_words3).csv",f"results_random_words3",10,1)'''

'#製作一般的timeframe = 10, space = 1 的results\nx=8\nq=x\ncalculate(f"data/Text/00{x}(done_random_words3).csv",f"results_random_words3",10,1)'

## 一般切割SD

In [32]:
'''#這個切半個週期SD用的
x=8
q=x
freq = float(input("enter freq: "))
period = 1/(freq/1000)
startt = input("enter start time: ")
endt= input("enter end time:")
#read_period(f"data/peaks_of_PSD/0{x}(peaks).csv")
cut_timeoffset = int(startt)
cut_timeend = int(endt)
phasenum=0
for t in range(cut_timeoffset,cut_timeoffset+int(period/2)): #這邊驅動的是不同phase的時間差
    # print("start_time =", t)
    phasenum+=1
    calculate(f"data/Text/00{x}(done_random_words2).csv",f"done_random_words2_halfpeakstep_phasef{math.floor(freq)}_{startt}_{endt}_{t-cut_timeoffset}",period/2,period/2,t,end = cut_timeend) #,
print("phase有:",phasenum)
print("freq=", freq)
print(startt, endt)
'''

'#這個切半個週期SD用的\nx=8\nq=x\nfreq = float(input("enter freq: "))\nperiod = 1/(freq/1000)\nstartt = input("enter start time: ")\nendt= input("enter end time:")\n#read_period(f"data/peaks_of_PSD/0{x}(peaks).csv")\ncut_timeoffset = int(startt)\ncut_timeend = int(endt)\nphasenum=0\nfor t in range(cut_timeoffset,cut_timeoffset+int(period/2)): #這邊驅動的是不同phase的時間差\n    # print("start_time =", t)\n    phasenum+=1\n    calculate(f"data/Text/00{x}(done_random_words2).csv",f"done_random_words2_halfpeakstep_phasef{math.floor(freq)}_{startt}_{endt}_{t-cut_timeoffset}",period/2,period/2,t,end = cut_timeend) #,\nprint("phase有:",phasenum)\nprint("freq=", freq)\nprint(startt, endt)\n'

## 地毯式頻率切割

In [33]:
#這個好像是切half套SD用的
#for x in range(10,50):
x=8
q=x

for freq in range(10,13):
    period = 1/(freq/1000)
    #read_period(f"data/peaks_of_PSD/0{x}(peaks).csv")
    for step in range(150,151,50):
        for startt in range(0,(481-step),10):
            cut_timeoffset = int(startt)
            endt = startt + step
            cut_timeend = int(endt)
            phasenum=0
            #print(startt, endt)
            for t in range(cut_timeoffset,cut_timeoffset+int(period/2)): #這邊驅動的是不同phase的時間差
                #print("start_time =", t)
                phasenum+=1
                calculate(f"data/Text/{x:0>3d}(done_random_words2).csv",f"random2_halfpeakstep_phasef{math.floor(freq)}_{startt}_{endt}_{t-cut_timeoffset}",period/2,period/2,t,end = cut_timeend)
            #print("phase有:",phasenu
            #print("freq=", freq)



KeyboardInterrupt: 

In [ ]:
'''x=8
q=x
for i in range(10,50):
    freq = i
    period = 1/(freq/1000)
    startt = 100
    endt= 480
    #read_period(f"data/peaks_of_PSD/0{x}(peaks).csv")
    cut_timeoffset = int(startt)
    cut_timeend = int(endt)
    phasenum=0
    for t in range(cut_timeoffset,cut_timeoffset+int(period/2)): #這邊驅動的是不同phase的時間差
        print("start_time =", t)
        phasenum+=1
        calculate(f"data/Text/00{x}(done).csv",f"halfpeakstep_phasef{math.floor(freq)}_{startt}_{endt}_{t-cut_timeoffset}",period/2,period/2,t,end = cut_timeend) #,
    print("phase有:",phasenum)
    print("freq=", freq)'''

start_time = 100
start_time = 101
start_time = 102
start_time = 103
start_time = 104
start_time = 105
start_time = 106
start_time = 107
start_time = 108
start_time = 109
start_time = 110
start_time = 111
start_time = 112
start_time = 113
start_time = 114
start_time = 115
start_time = 116
start_time = 117
start_time = 118
start_time = 119
start_time = 120
start_time = 121
start_time = 122
start_time = 123
start_time = 124
start_time = 125
start_time = 126
start_time = 127
start_time = 128
start_time = 129
start_time = 130
start_time = 131
start_time = 132
start_time = 133
start_time = 134
start_time = 135
start_time = 136
start_time = 137
start_time = 138
start_time = 139
start_time = 140
start_time = 141
start_time = 142
start_time = 143
start_time = 144
start_time = 145
start_time = 146
start_time = 147
start_time = 148
start_time = 149
phase有: 50
freq= 10
start_time = 100
start_time = 101
start_time = 102
start_time = 103
start_time = 104
start_time = 105
start_time = 106
start_time 

gcs,destination,timeframe,space,start_time=0,end=480

## 這個好像是切整套SD用的

In [ ]:
#這個好像是切整套SD用的
'''x=8
q=x
freq = int(input("enter freq: "))
period = float(1/(freq/1000))
startt = input("enter start time: ")
endt= input("enter end time:")
#read_period(f"data/peaks_of_PSD/0{x}(peaks).csv")
cut_timeoffset = int(startt)
cut_timeend = int(endt)
phasenum=0
for t in range(cut_timeoffset,cut_timeend-int(period/2)): #這邊驅動的是不同phase的時間差
    print("start_time =", t)
    phasenum+=1
    calculate(f"data/Text/00{x}(done_shuffled).csv",f"shuffled_halfpeakstep_phasef{math.floor(freq)}_{startt}_{endt}_{t-cut_timeoffset}",period/2,period/2,t,end = cut_timeend) #,
print("phase有:",phasenum)
print("freq=", freq)'''

'x=8\nq=x\nfreq = int(input("enter freq: "))\nperiod = float(1/(freq/1000))\nstartt = input("enter start time: ")\nendt= input("enter end time:")\n#read_period(f"data/peaks_of_PSD/0{x}(peaks).csv")\ncut_timeoffset = int(startt)\ncut_timeend = int(endt)\nphasenum=0\nfor t in range(cut_timeoffset,cut_timeend-int(period/2)): #這邊驅動的是不同phase的時間差\n    print("start_time =", t)\n    phasenum+=1\n    calculate(f"data/Text/00{x}(done_shuffled).csv",f"shuffled_halfpeakstep_phasef{math.floor(freq)}_{startt}_{endt}_{t-cut_timeoffset}",period/2,period/2,t,end = cut_timeend) #,\nprint("phase有:",phasenum)\nprint("freq=", freq)'

In [ ]:
'''
start_subject = int(input("start_subject = "))
end_subject = int(input("end_subject = "))
for x in range(start_subject,end_subject+1):
    if x<10:
        try: 
            q=x
            read_period(f"data/peaks_of_PSD/00{x}(peaks).csv")
            for t in range(0,int(period/2)):
                print("start_time =", t)
                calculate(f"data/Text/00{x}(done).csv",f"halfpeakstep_phase0f{t}",period/2,period/2,t)
        except:
            print("error:",x)
            continue
        
    else:
        q=x
        try:
            read_period(f"data/peaks_of_PSD/0{x}(peaks).csv")
            for t in range(0,int(period/2)):
                print("start_time =", t)
                calculate(f"data/Text/0{x}(done).csv",f"halfpeakstep_phase0f{t}",period/2,period/2,t)
        except:
            print("error:",x)
            continue
'''

'\nstart_subject = int(input("start_subject = "))\nend_subject = int(input("end_subject = "))\nfor x in range(start_subject,end_subject+1):\n    if x<10:\n        try: \n            q=x\n            read_period(f"data/peaks_of_PSD/00{x}(peaks).csv")\n            for t in range(0,int(period/2)):\n                print("start_time =", t)\n                calculate(f"data/Text/00{x}(done).csv",f"halfpeakstep_phase0f{t}",period/2,period/2,t)\n        except:\n            print("error:",x)\n            continue\n        \n    else:\n        q=x\n        try:\n            read_period(f"data/peaks_of_PSD/0{x}(peaks).csv")\n            for t in range(0,int(period/2)):\n                print("start_time =", t)\n                calculate(f"data/Text/0{x}(done).csv",f"halfpeakstep_phase0f{t}",period/2,period/2,t)\n        except:\n            print("error:",x)\n            continue\n'

In [ ]:

start_subject = int(input("start_subject = "))
end_subject = int(input("end_subject = "))
for x in range(start_subject,end_subject+1):
    if x<10:
        try: 
            q=x
            #read_period(f"data/peaks_of_PSD/00{x}(peaks).csv")
            calculate(f"data/Text/00{x}(done).csv","step0303",3,3)
        except:
            print("error:",x)
            continue
        
    else:
        q=x
        try:
           #read_period(f"data/peaks_of_PSD/0{x}(peaks).csv")
            calculate(f"data/Text/0{x}(done).csv","step0303",3,3)
        except:
            print("error:",x)
            continue


ValueError: invalid literal for int() with base 10: ''

In [ ]:
print(len(all_sentences001))
print(len(SBERT))

160
159


In [ ]:
'''
for i in range(18,24):
    if i<10:
        try:
            calculate(f"data/Text/00{i}(done).csv")
            results_csv = pd.DataFrame(list(zip(SBERT,all_sentences_wu_sum,all_sentences_wu_sum_divided,pair_counts)), columns = ['SBERT',"Wu-Pulmer","Wu-Pulmer_divided","pair_counts"])
            results_csv.to_csv(f'data/Text/00{i}(results).csv')
        except:
            print("error:", i)
            continue
    else:
        try:
            calculate(f"data/Text/0{i}(done).csv")
            results_csv = pd.DataFrame(list(zip(SBERT,all_sentences_wu_sum,all_sentences_wu_sum_divided,pair_counts)), columns = ['SBERT',"Wu-Pulmer","Wu-Pulmer_divided","pair_counts"])
            results_csv.to_csv(f'data/Text/0{i}(results).csv')
        except:
            print("error:", i)
            continue
'''

'\nfor i in range(18,24):\n    if i<10:\n        try:\n            calculate(f"data/Text/00{i}(done).csv")\n            results_csv = pd.DataFrame(list(zip(SBERT,all_sentences_wu_sum,all_sentences_wu_sum_divided,pair_counts)), columns = [\'SBERT\',"Wu-Pulmer","Wu-Pulmer_divided","pair_counts"])\n            results_csv.to_csv(f\'data/Text/00{i}(results).csv\')\n        except:\n            print("error:", i)\n            continue\n    else:\n        try:\n            calculate(f"data/Text/0{i}(done).csv")\n            results_csv = pd.DataFrame(list(zip(SBERT,all_sentences_wu_sum,all_sentences_wu_sum_divided,pair_counts)), columns = [\'SBERT\',"Wu-Pulmer","Wu-Pulmer_divided","pair_counts"])\n            results_csv.to_csv(f\'data/Text/0{i}(results).csv\')\n        except:\n            print("error:", i)\n            continue\n'